# Figure 5

This notebook recreates the figure panels included in Figure 5 of Lee et al. 2021. 



### Description of the Data

The data used in this notebook comes from the experiments described in Lee et al. 2021. Specifically, we have the behavioral and activity of a trained deep RL agent performing a evidence accumulation task from Engelhard et al. 2019. 




## Preparing the Data
### Importing required code packages and modules

In [1]:
ls

Figure 2.ipynb       Figure 5.ipynb       Figure 8.ipynb
Figure 3.ipynb       Figure 6.ipynb       Supp Figures.ipynb
Figure 3_OLD.ipynb   Figure 7.ipynb       utils/
Figure 4.ipynb       Figure 7_supp.ipynb


In [1]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import sys
from scipy.io import loadmat, savemat
import utils.cnnlstm_analysis_utils as utils
import seaborn as sns
from scipy import stats
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
import os

Missing colon in file '/Users/sasha/.matplotlib/matplotlibrc', line 1 ('~/.matplotlib/matplotlibrc')


In [3]:
ls

Figure 2.ipynb                  Figure 8.ipynb
Figure 3.ipynb                  Supp Figures.ipynb
Figure 3_OLD.ipynb              frames_analysis.ipynb
Figure 4.ipynb                  mean_neural_activity.mat
Figure 5.ipynb                  mean_neural_activity_by1cm.mat
Figure 6.ipynb                  sig_neurons.mat
Figure 7.ipynb                  utils/
Figure 7_supp.ipynb


### downloading data

In [4]:
load_prefix = '../data/logs/VA_maze/'
with open(load_prefix + '5000t_mosttrain_db.p', 'rb') as f:
     [actions_, rewards_, feats_, terms_, vs_, tow_counts_, episode_lengths]  = pickle.load(f)
f.close()
vs = np.hstack(vs_)
terms = np.hstack(terms_)
rewards = np.hstack(rewards_)
ep_rew = np.array([np.sum(r_trial) for r_trial in rewards_])
ep_tow = np.array([np.max(trial, 0) for trial in tow_counts_])
tow_counts = np.vstack(tow_counts_)



weights = utils.get_params_from_zip(load_prefix + 'rl_model_20800000_steps')
# weights.keys()

w_pol = weights['model/pi/w:0']
b_pol = weights['model/pi/b:0']
w_val = np.squeeze(weights['model/vf/w:0'])
b_val = weights['model/vf/b:0']

trial_info = loadmat(load_prefix + 'trialinfo_db.mat')

trial_info = trial_info['trials']
trial_info.dtype.names
choices = utils.extract_field(trial_info, 'choice')
trial_type = utils.extract_field(trial_info, 'trialType')
raw_ypos = utils.extract_field(trial_info, 'position')[:,1]
cueCombos_ = utils.extract_field(trial_info, 'cueCombo')
cuePos_ = utils.extract_field(trial_info, 'cuePos')
cueOnset_ = utils.extract_field(trial_info, 'cueOnset')
raw_ypos_ = [x[:,1] for x in trial_info['position'][0]]
raw_xpos_ = [x[:,0] for x in trial_info['position'][0]]
raw_vpos_ = [x[:,2] for x in trial_info['position'][0]]
ypos_ = [np.hstack([np.array(x[:-1]), x[-2] * np.ones((7,))]) for x in raw_ypos_]
ypos = np.hstack(ypos_)


### Plotting Parameters

In [5]:
# PLOTTING PARAMS
matplotlib.rcParams.update({'font.size': 15})
matplotlib.rcParams.update({'font.family': 'Arial'})

FONT_BG = 25
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # allow text of pdf to be edited in illustrator

mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False

marker_plots =  {'marker':'.', 'markersize':2, 'markeredgecolor':'k', 'markerfacecolor':'k'}


### Organizing Data

Pulling out the specific data that we will use for figure panels

In [6]:
CUEP_LIM = 140
REWP_LEN_S = -16
REWP_LEN_STP = -5
ypos_cuep = np.squeeze(np.dstack([ypos_t[:CUEP_LIM] for ypos_t in ypos_])[:,:,0])
(ep_towdelt_idx, ep_towdiff_idx) = utils.get_ep_tow_idx(ep_tow)
ep_rewidx = (episode_lengths - 11).astype(int)


### Calculate Vector RPEs

In [7]:
if os.path.exists(load_prefix + 'pes.p'):
    with open(load_prefix + 'pes.p', 'rb') as f:
         pes  = pickle.load(f)
    f.close()

else: 
    feats = np.vstack(feats_)
    rewards = np.hstack(rewards_)
    terms = np.hstack(terms_)
    start = np.roll(terms,1)
    nsteps = len(terms)
    nfeatures = feats_[0][0].shape[0]
    gamma = 0.99
    # compute per-feature PEs
    pes = np.zeros((nsteps, nfeatures))
    for i in range(0,nsteps-1):
        if (terms[i]): # there is a one-off error-- the SECOND index of the start of the trial accurately measures the start of the trial
            pes[i,:] = rewards[i] / nfeatures -  w_val * feats[i,:]
        else:
            pes[i,:] = rewards[i] / nfeatures +  w_val * (-feats[i,:] + gamma * feats[i+1,:])

    pickle.dump(pes, open(load_prefix + "pes.p", "wb") )




In [8]:

# summing pes to get scalar
pes_sum = np.sum(pes,1)
# scalar pes binned by position
pes_sum_ = utils.split_by_ep_len(pes_sum, np.hstack((episode_lengths)))
# pes split by 5000 trials
pes_ = utils.split_by_ep_len(pes, np.hstack((episode_lengths)))

pes_sum_rewp = np.vstack([trial[rew_idx - 5:rew_idx + 6] for [trial, rew_idx] in zip(pes_sum_, ep_rewidx)])
pes_rewp = np.dstack([pes_i[rew_idx - 5:rew_idx + 6,:] for [pes_i, rew_idx] in zip(pes_, ep_rewidx)])
vs_rewp = np.vstack([pes_i[rew_idx - 5:rew_idx + 6] for [pes_i, rew_idx] in zip(vs_, ep_rewidx)])

pes_rewp_norm = utils.norm_within_feat(pes_rewp.transpose((1, 0, 2)).reshape(64, 11 * 5000)).reshape(64, 11, 5000).transpose(1, 0, 2)

In [9]:
pes_rewp_norm.shape

(11, 64, 5000)

In [10]:
# EASY VS HARD TRIALS AND RESPONSES

rew_trials = np.where(ep_rew == 1)[0]
nrew_trials = np.where(ep_rew == 0)[0]
easy_trials = np.hstack(ep_towdelt_idx[5:])
hard_trials = np.hstack(ep_towdelt_idx[:2])

rew_hard = np.sort(list(set(rew_trials) & set(hard_trials)))
rew_easy = np.sort(list(set(rew_trials) & set(easy_trials)))

norm_easy = (np.mean(pes_rewp_norm[:,:,rew_easy], -1).T)[:,5]
norm_hard = (np.mean(pes_rewp_norm[:,:,rew_hard], -1).T)[:,5]

norm_nrew = (np.mean(pes_rewp_norm[:,:,nrew_trials], -1).T)[:,5]
norm_rew = (np.mean(pes_rewp_norm[:,:,rew_trials], -1).T)[:,5]

In [13]:
# neural data uses matlab function: outcome_dprime.m

neural_diff = loadmat('../../data/ben_dprime_all.mat')['diff'][0,:]
neural_rw = loadmat('../../data/ben_dprime_all.mat')['currw'][0,:]


## Plot figure

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = gridspec.GridSpec(2, 3, width_ratios = [2, 1, 1])


# Figure 5a: Reward Scalar RPE
ax_sum = plt.subplot(gs[0,0])
ax_sum.plot(np.arange(-5,6), np.sum(np.nanmean(pes_rewp[:,:,ep_rew == 1],-1),-1),
                                    **marker_plots, color = 'mediumvioletred', label = 'Rewarded');
ax_sum.plot(np.arange(-5,6), np.sum(np.nanmean(pes_rewp[:,:,ep_rew == 0],-1),-1),
                                    **marker_plots, color = 'gray', label = 'Unrewarded');
ax_sum.axvline(0, color = 'grey', alpha = 0.8, lw = 0.5, ls = ':')
ax_sum.axvline(0,  ls = (0,(5,5)), color = 'k', lw = 0.75)
ax_sum.set_ylabel('Scalar RPE')
ax_sum.set_xlabel('Time From Reward')
ax_sum.set_title('Model\n', fontsize = 15)
ax_sum.legend(frameon=False)


# Figure 5b: Reward Vector RPEs Histograms
ax_hist =  plt.subplot(gs[0, 1])


_, uniform_bins, _ = ax_hist.hist(norm_rew - norm_nrew,color = 'grey',
                    bins = 30,  range = (-np.max(neural_rw), np.max(neural_rw)));
ax_hist.axvline(np.median(norm_rew - norm_nrew), color = 'gold')
ax_hist.axvline(0,  ls = (0,(5,5)), color = 'k', lw = 0.75)
ax_hist.set_xlim(xmin = -np.max(neural_rw), xmax = np.max(neural_rw))
pval_rw = stats.wilcoxon(norm_rew - norm_nrew)
ax_hist.set_title('Model\nP < ' + str(max(pval_rw.pvalue, 5e-12)), fontsize = 15)
ax_hist.set_xlabel('\t \t \t \t \t \t \t Reward Trials - Unrewarded Trials'.expandtabs())
ax_hist.set_ylabel('Vector RPEs')


# Figure 5c: Reward Neurons Histograms 
ax_neural =  plt.subplot(gs[0,-1])
ax_neural.hist(neural_rw, color = 'gray', bins = uniform_bins)
ax_neural.axvline(np.mean(neural_rw), color = 'gold')
ax_neural.axvline(0, color = 'k', ls = (0,(5,5)), lw = 0.75)
ax_neural.set_xlim(xmin = -np.max(neural_rw))
pval_rw_neural = stats.wilcoxon(neural_rw)
ax_neural.set_title('Neural Data\nP < ' + str(max(pval_rw_neural.pvalue, 1e-48)), fontsize = 15)
ax_neural.set_ylabel('Neurons')


# Figure 5d: Reward Difficulty Scalar RPE 
ax_sum = plt.subplot(gs[1,0])
ax_sum.plot(np.arange(-5,6),np.sum(np.nanmean(pes_rewp[:,:,rew_hard],-1),-1),
                                **marker_plots, color = 'skyblue', label = 'Hard');
ax_sum.plot(np.arange(-5,6),np.sum(np.nanmean(pes_rewp[:,:,rew_easy],-1),-1),
                                **marker_plots, color = 'mediumblue', label = 'Easy');
ax_sum.axvline(0, color = 'grey', alpha = 0.8, lw = 0.5, ls = ':')
ax_sum.set_ylabel('Scalar RPE')
ax_sum.set_xlabel('Time From Reward')
ax_sum.set_title('Model\n', fontsize = 15)
ax_sum.legend(frameon=False, loc = 'upper left')

# Figure 5e: Reward Difficulty Vector RPE Histograms
ax_hist =  plt.subplot(gs[1,1])
ax_hist.hist(norm_hard - norm_easy, color = 'grey', bins = 30, range = (-0.003, 0.003));
ax_hist.axvline(np.median(norm_hard - norm_easy), color = 'gold');
ax_hist.axvline(0,  ls = (0,(5,5)), color = 'k', lw = 0.75);
pval_diff = stats.wilcoxon(norm_hard - norm_easy)
ax_hist.set_title('Model\nP < ' + str(max(pval_diff.pvalue, 0.05)), fontsize = 15)
ax_hist.set_xlabel('\t \t \t \t \t \t \t Hard Trials - Easy Trials'.expandtabs())
ax_hist.set_ylabel('Vector RPEs')

# Figure 5f: Reward Difficulty Neurons Histograms 
ax_neural =  plt.subplot(gs[1,-1])
ax_neural.hist(neural_diff, color = 'gray', bins = 30, range = (-0.05, 0.05))
ax_neural.axvline(np.median(neural_diff), color = 'gold')
ax_neural.axvline(0, color = 'k', ls = (0,(5,5)), lw = 0.75)
pval_diff_neural = stats.wilcoxon(neural_diff)
ax_neural.set_title('Neural Data\nP < ' + str(max(pval_diff_neural.pvalue, 3e-5)), fontsize = 15)
ax_neural.set_ylabel('Neurons')


plt.tight_layout(w_pad = 1.3)


In [11]:
stats.wilcoxon(norm_rew - norm_nrew)


WilcoxonResult(statistic=5.0, pvalue=4.466736295633894e-12)

In [16]:
stats.wilcoxon(neural_rw)

WilcoxonResult(statistic=436.0, pvalue=1.4309823815262544e-49)

In [12]:
stats.wilcoxon(norm_hard - norm_easy)

WilcoxonResult(statistic=721.0, pvalue=0.03289897651981216)

In [17]:
stats.wilcoxon(neural_diff)

WilcoxonResult(statistic=16652.0, pvalue=2.949173825044694e-05)